# Bayesian Optimization
Idea: Use bayesian optimization for (wrapper) feature selection.

In [155]:
from sklearn import datasets
from sklearn import svm
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from bayes_opt import BayesianOptimization
from itertools import chain, combinations
from skopt import gp_minimize

## 0. Hyperparameters

In [156]:
k_features = 3 # number of features which we want to get

## 1. Choose dataset

In [157]:
# regression dataset
dataset = datasets.load_boston()

In [134]:
# classification dataset
dataset = datasets.load_wine()

## 2. Import dataset

In [158]:
# define which dataset should be used
data = dataset.data
data_target = dataset.target
feature_names = dataset.feature_names

data_frame = pd.DataFrame(data, columns = feature_names)
data_frame['Target'] = data_target
X = data_frame.drop("Target", 1)       # feature matrix
y = data_frame['Target'] 
data_frame.head()

#print(data)
#print(target_matrix)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## 3. Forward selection Algorithm

In [159]:
forward_selection = SFS(LinearRegression(),
          k_features=k_features,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)

In [160]:
forward_selection.fit(X, y)
forward_selection.k_feature_names_ 

('RM', 'PTRATIO', 'LSTAT')

## 4. Create feature-subsets (wrapper)

In [161]:
s = feature_names 
wrapper_temp = list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))

# filter depending on hyperparameter 'k_features'
wrapper = []
for element in wrapper_temp:
    if len(element) == k_features:
        wrapper.append(element)

## 5. Bayesian Optimization Algorithm

### 5.1  fmfn / BayesianOptimization Package 
- https://github.com/fmfn/BayesianOptimization

In [162]:
def black_box_function_bay_opt(x):
    x = round(x)
    filteredX = X[X.columns[X.columns.isin(wrapper[x])]] # use only selected features
    model = LinearRegression().fit(filteredX, y)
    return model.score(filteredX, y)


In [163]:
pbounds = {'x': (1, len(wrapper)-1)}

In [164]:
optimizer_fmfn = BayesianOptimization(
    f=black_box_function_bay_opt,
    pbounds=pbounds,
    verbose=1, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=123,
)

In [165]:
optimizer_fmfn.maximize(
    init_points=2,
    n_iter=100,
)

|   iter    |  target   |     x     |
-------------------------------------
|  3        |  0.6185   |  200.1    |
|  21       |  0.64     |  144.2    |
|  41       |  0.6468   |  240.1    |
|  46       |  0.6485   |  246.7    |
|  48       |  0.6786   |  249.2    |


### 5.2 scikit-optimize (skopt)
- https://scikit-optimize.github.io/stable/auto_examples/bayesian-optimization.html

In [166]:
def black_box_function_scikit_optimize(x):
    x = round(x[0])
    
    filteredX = X[X.columns[X.columns.isin(wrapper[x])]] # use only selected features
    model = LinearRegression().fit(filteredX, y)
    return 1-model.score(filteredX, y)


In [168]:
optimizer_skopt = gp_minimize(black_box_function_scikit_optimize, # the function to minimize
                  [(1, len(wrapper)-1)],      # the bounds on each dimension of x
                  acq_func="EI",      # the acquisition function
                  n_calls=100,         # the number of evaluations of f
                  n_random_starts=2,  # the number of random initialization points
                  #noise=0.1**2,       # the noise level (optional)
                  random_state=123,  # the random seed
                  verbose=False)  

print(1-optimizer_skopt.fun)
print(optimizer_skopt.x)

/home/patrick/.local/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/patrick/.local/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/patrick/.local/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/patrick/.local/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/patrick/.local/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has be

0.6505548381286019
[250]


## 6. Results

In [169]:
# forward selection algorithm
forward_selection.k_feature_names_ 

('RM', 'PTRATIO', 'LSTAT')

In [170]:
# fmfn bayesian optimization
print(wrapper[round(optimizer_fmfn.max["params"]["x"])])

('RM', 'PTRATIO', 'LSTAT')


In [171]:
# skopt bayesian optimization
print(wrapper[round(optimizer_skopt.x[0])])

('RM', 'B', 'LSTAT')
